<img src="../Images/DSC_Logo.png" style="width: 400px;">

# Time Series Concepts in Python 

Basic Python packages for data handling:

In [ ]:
import numpy as np
import pandas as pd

Python packages for specific analysis:

In [ ]:
import math
import statistics as stat
import random

Matplotlib - the basic library for plotting:

In [ ]:
import matplotlib.pyplot as plt

Python packages specifically for time series data analysis:

In [ ]:
from datetime import datetime

## 1. Create time series in Python

In [ ]:
my_year = 2020
my_month = 1
my_day = 23
#my_hour = 13
#my_min = 30
#my_sec = 15
my_date = datetime(my_year,my_month,my_day)

In [ ]:
my_date

In [ ]:
my_date.day

Operations in datetime format:

In [ ]:
nowday = datetime.today()
diff = nowday - my_date
diff.days

## 1.2. Create DateTime object using NumPy

In [ ]:
np.array(['2020-03-09','2020-03-10','2020-03-11','2020-03-12','2020-03-13','2020-03-14'], dtype='datetime64')
# or:
np.arange('2020-03-09','2020-03-15', dtype='datetime64[D]')

## 1.3. Create DateTime object using Pandas

In [ ]:
pd.date_range('2020-03-09', periods=6, freq='D')

Specify format:

In [ ]:
pd.to_datetime('2020-09-03', format='%Y-%d-%m')

**Exercise:** Test how the following format could be converted in a DateTime object: 15/10/2020

## 1.4. Create a Pandas time series dataframe

In [ ]:
data = np.random.randn(6,2) # create some random data
data

In [ ]:
cols = ['A','B']
idx = pd.date_range('2020-03-09', periods=6, freq='D')
df = pd.DataFrame(data, index=idx, columns=cols)
df

In [ ]:
df.plot()

**Exercise:** Create a Pandas time series dataframe that contains decadal data.

## 1.5 Interchange date column and index in dataframes

In [ ]:
data = np.random.randn(100,2) # create some random data
cols = ['A','B']
idx = pd.date_range('2020-03-09', periods=100, freq='D')
df = pd.DataFrame(data, index=idx, columns=cols)

df['Date'] = df.index
df

## 1.6 Derive DateTime elements

In [ ]:
df['Month'] = df['Date'].dt.month
df

In [ ]:
df['Month'] = df.index.month
df

**Exercise:** How could we define a new column with the years?

## 2. Basic time series operations

## 2.1. Resampling - changing the frequency of time series data

In [ ]:
df.resample('W-MON').mean()

## 2.2 Shifting the data points in a time series forward (or backward) in time

In [ ]:
df.shift(1)

## 2.3 Time-based slicing

In [ ]:
df['2020-03-10':'2020-03-12']

## 3. Import & analyze - temperature time series
![sky](../Images/temperature.jpg)

*Image modified from Gerd Altmann, Pixabay*

## 3.1. Pandas for csv files

Import data from csv:

In [ ]:
path = '../Datasets/NOAA_time_series.csv' 
df = pd.read_csv(path, skiprows=4, delimiter=',')

In [ ]:
df.head()

Summary statistics:

In [ ]:
df.describe()

Setting the time variable as index:

In [ ]:
df.set_index('Year', inplace=True) # Set the 'Date' column as the index

In [ ]:
df.head()

Plot:

In [ ]:
df.plot()

### More plots ...

### Boxplot:

In [ ]:
# Create a new column for decade
df['Decade'] = (df.index // 10) * 10

# Prepare data for boxplot
decade_data = [df[df['Decade'] == decade]['Anomaly'].values for decade in df['Decade'].unique()]

# Create boxplot using matplotlib
plt.figure()
plt.boxplot(decade_data, labels=df['Decade'].unique())
plt.title('Decade Boxplot of Anomalies')
plt.xlabel('Decade')
plt.ylabel('Anomaly')
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.show()

### Bar plot:

In [ ]:
# Calculate the average anomaly per decade
average_anomalies = df.groupby('Decade')['Anomaly'].median()

# Create a bar plot using matplotlib
plt.figure()
average_anomalies.plot(kind='bar', color='skyblue')
plt.title('Average Anomalies by Decade')
plt.xlabel('Decade')
plt.ylabel('Average Anomaly')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

## 3.2. xarray for netCDF

Like pandas for netCDF ... common data format for time series data ...

In [ ]:
pip install xarray

In [ ]:
import xarray as xr

Import a dataset from [ERA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels) reanalysis: 

In [ ]:
ERA5 = xr.open_dataset('../Datasets/ERA5_snippet.nc')

Investigate:

In [ ]:
ERA5

In [ ]:
ERA5['tp']

In [ ]:
ERA5['t2m']

Mathematical operations:

In [ ]:
ERA5['t2m'].values = (ERA5.t2m-273.15).values

In [ ]:
ERA5

Resample:

In [ ]:
ERA5 = ERA5.resample(time='1D').mean()  # Daily mean

In [ ]:
ERA5

Select a specific date:

In [ ]:
ERA5['t2m'][0,:,:] # select by index: first entry

In [ ]:
ERA5['t2m'].sel(time='2023-07-01') # select by date

Plot with matplotlib:

In [ ]:
ERA5['t2m'][0,:,:].plot()

Plotting on a map using Cartopy & Matplotlib:

In [ ]:
pip install cartopy

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Use the plot method for quick visualization
ERA5['t2m'][0,:,:].plot(ax=ax, cmap='coolwarm', add_colorbar=True)

# Add coastlines and grid lines
ax.coastlines()
ax.gridlines(draw_labels=True)

plt.title('2-Meter Temperature [°C] on 2023-01-01 00:00:00 over Germany')
plt.show()

### Temperature time series for Bremen (single grid cell):

Select data:

In [ ]:
# Select a specific location (e.g., longitude = 10.0, latitude = 50.0)
bremen = ERA5.sel(longitude=8.808, latitude=53.075, method='nearest')

In [ ]:
bremen

Plot:

In [ ]:
plt.figure(figsize=(10, 5))
bremen['t2m'].plot()
plt.title('Daily mean temperature for Bremen in the year 2023')
plt.ylabel('Temperature (°C)')
plt.xlabel('Date')
plt.grid()
plt.show()

**Exercise:** Select and plot the temperature time series for another region of your choice in Germany.